# Hybrid RAG Supply Chain Copilot (FAISS + Local Llama)

This notebook is a self-contained, safe-to-share demo:
- Synthetic supply-chain tables
- Rule-based retrieval for structured signals
- FAISS semantic retrieval for supplier notes (falls back to TF-IDF)
- Hybrid retrieval fusion
- Deterministic risk engine
- Local Llama (Ollama) wrapper + strict JSON validation + safe fallback


In [1]:
# Core imports + reproducibility
import numpy as np
import pandas as pd
import random
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple, Optional

random.seed(42)
np.random.seed(42)

pd.set_option("display.width", 120)
pd.set_option("display.max_colwidth", 80)


## 1) Synthetic data model

In [2]:
# Synthetic SKU universe
n_skus = 30
skus = [f"SKU-{i:03d}" for i in range(1, n_skus + 1)]

# inventory
inventory = pd.DataFrame({
    "sku_id": skus,
    "on_hand_units": np.random.randint(20, 600, size=n_skus),
    "avg_daily_demand": np.random.randint(5, 80, size=n_skus),
    "safety_stock_units": np.random.randint(20, 150, size=n_skus),
})
inventory["days_of_cover"] = (inventory["on_hand_units"] / inventory["avg_daily_demand"]).round(1)

# demand signals
demand_signals = pd.DataFrame({
    "sku_id": skus,
    "demand_spike_pct": np.random.choice([0, 0, 0, 10, 20, 35, 50, 75], size=n_skus),
    "promo_flag": np.random.choice([0, 1], size=n_skus, p=[0.75, 0.25]),
})

# shipments (delays)
shipments = pd.DataFrame({
    "sku_id": skus,
    "max_delay_days": np.random.randint(0, 15, size=n_skus),
    "eta_days": np.random.randint(1, 21, size=n_skus),
})

# purchase orders
purchase_orders = pd.DataFrame({
    "sku_id": skus,
    "open_po_units": np.random.randint(0, 800, size=n_skus),
    "buyer_priority": np.random.choice(["normal", "high", "critical"], size=n_skus, p=[0.65, 0.25, 0.10]),
})

# supplier notes (text)
note_templates = [
    "Supplier reported a raw material shortage affecting lead times.",
    "Carrier congestion at port; containers delayed by {d} days.",
    "Quality hold on latest batch; inspection pending.",
    "Customer promotion increased demand unexpectedly.",
    "Partial shipment confirmed; remaining units backordered.",
    "No issues reported; stable supply.",
    "Factory maintenance shutdown planned next week.",
    "Expedite request submitted; awaiting confirmation.",
    "Weather disruption reported in transit corridor.",
]

rows = []
note_id = 1
for sku in skus:
    # 0 to 2 notes per SKU
    k = np.random.choice([0, 1, 2], p=[0.25, 0.5, 0.25])
    for _ in range(int(k)):
        tmpl = random.choice(note_templates)
        d = int(np.random.randint(1, 14))
        text = tmpl.format(d=d)
        rows.append({"note_id": f"N{note_id:04d}", "sku_id": sku, "note_text": text})
        note_id += 1

supplier_notes = pd.DataFrame(rows)

# Features table for retrieval + risk scoring
features = (
    inventory
    .merge(demand_signals, on="sku_id", how="left")
    .merge(shipments, on="sku_id", how="left")
    .merge(purchase_orders, on="sku_id", how="left")
)

# A simple projected net availability after 7 days (very rough, for demo only)
# net_available_7d = on_hand + open_po - 7*avg_daily_demand
features["net_available_7d"] = features["on_hand_units"] + features["open_po_units"] - 7 * features["avg_daily_demand"]

# A basic stockout flag proxy for retrieval weighting
features["stockout_risk_flag_7d"] = (features["net_available_7d"] < 0).astype(int)

features.head(8)


,sku_id,on_hand_units,avg_daily_demand,safety_stock_units,days_of_cover,demand_spike_pct,promo_flag,max_delay_days,eta_days,open_po_units,buyer_priority,net_available_7d,stockout_risk_flag_7d
0,SKU-001,122,46,69,2.7,0,1,8,20,510,normal,310,0
1,SKU-002,455,64,123,7.1,20,0,6,17,751,normal,758,0
2,SKU-003,290,19,21,15.3,0,0,13,17,143,critical,300,0
3,SKU-004,126,66,73,1.9,50,0,0,20,608,normal,272,0
4,SKU-005,91,66,125,1.4,0,0,0,12,200,normal,-171,1
5,SKU-006,40,51,23,0.8,0,0,8,7,123,critical,-194,1
6,SKU-007,141,66,73,2.1,10,0,10,2,186,critical,-135,1
7,SKU-008,486,55,63,8.8,50,0,13,3,325,normal,426,0


## 2) Rule-based retrieval (structured signals)

In [3]:
def rule_retrieval(features: pd.DataFrame, question: str, top_k: int = 10) -> pd.DataFrame:
    q = question.lower()
    w_stockout = 1.0 if "stockout" in q else 0.7
    w_delay = 1.0 if ("delay" in q or "shipment" in q) else 0.7
    w_demand = 1.0 if ("demand" in q or "promo" in q) else 0.8

    df = features.copy()
    df["retrieval_score_rule"] = (
        w_stockout * (df["stockout_risk_flag_7d"] * 50)
        + (df["days_of_cover"] < 7).astype(int) * 20
        + (df["days_of_cover"] < 4).astype(int) * 20
        + w_delay * df["max_delay_days"].clip(0, 10) * 4
        + w_demand * (df["demand_spike_pct"].clip(0, 100) / 5)
        + df["buyer_priority"].map({"normal": 0, "high": 5, "critical": 10}).fillna(0)
    )
    return df.sort_values("retrieval_score_rule", ascending=False).head(top_k)

question = "Which SKUs are at risk of stockout next week, and why?"
rule_top = rule_retrieval(features, question, top_k=10)[[
    "sku_id","days_of_cover","max_delay_days","demand_spike_pct","buyer_priority",
    "stockout_risk_flag_7d","retrieval_score_rule"
]]
rule_top


,sku_id,days_of_cover,max_delay_days,demand_spike_pct,buyer_priority,stockout_risk_flag_7d,retrieval_score_rule
6,SKU-007,2.1,10,10,critical,1,129.6
5,SKU-006,0.8,8,0,critical,1,122.4
14,SKU-015,1.9,5,50,normal,1,112.0
4,SKU-005,1.4,0,0,normal,1,90.0
29,SKU-030,2.0,8,35,normal,0,68.0
25,SKU-026,0.9,14,0,normal,0,68.0
11,SKU-012,1.9,8,0,high,0,67.4
0,SKU-001,2.7,8,0,normal,0,62.4
18,SKU-019,6.8,13,20,high,0,56.2
13,SKU-014,4.8,6,75,high,0,53.8


## 3) Semantic retrieval over supplier notes (FAISS if available, otherwise TF-IDF fallback)

In [ ]:
from typing import Callable

def build_semantic_backend(supplier_notes: pd.DataFrame) -> Tuple[str, Callable[[str,int], pd.DataFrame]]:
    # Try FAISS + sentence-transformers. If unavailable, fall back to TF-IDF.
    try:
        import faiss
        from sentence_transformers import SentenceTransformer

        if supplier_notes.empty:
            def _empty(query: str, top_k_notes: int = 6) -> pd.DataFrame:
                return pd.DataFrame(columns=["note_id","sku_id","note_text","semantic_score"])
            return "faiss(empty)", _empty

        texts = supplier_notes["note_text"].tolist()
        embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
        note_embeddings = embed_model.encode(texts, normalize_embeddings=True)
        note_embeddings = np.array(note_embeddings, dtype="float32")

        index = faiss.IndexFlatIP(note_embeddings.shape[1])
        index.add(note_embeddings)

        def semantic_note_retrieval(query: str, top_k_notes: int = 6) -> pd.DataFrame:
            q_emb = embed_model.encode([query], normalize_embeddings=True)
            q_emb = np.array(q_emb, dtype="float32")
            scores, idxs = index.search(q_emb, top_k_notes)

            rows = []
            for score, idx in zip(scores[0], idxs[0]):
                note = supplier_notes.iloc[int(idx)]
                rows.append({
                    "note_id": note["note_id"],
                    "sku_id": note["sku_id"],
                    "note_text": note["note_text"],
                    "semantic_score": float(score)
                })
            return pd.DataFrame(rows)

        return "faiss", semantic_note_retrieval

    except Exception:
        # TF-IDF fallback
        from sklearn.feature_extraction.text import TfidfVectorizer
        from sklearn.metrics.pairwise import cosine_similarity

        if supplier_notes.empty:
            def _empty(query: str, top_k_notes: int = 6) -> pd.DataFrame:
                return pd.DataFrame(columns=["note_id","sku_id","note_text","semantic_score"])
            return "tfidf(empty)", _empty

        texts = supplier_notes["note_text"].tolist()
        vectorizer = TfidfVectorizer(stop_words="english")
        X = vectorizer.fit_transform(texts)

        def semantic_note_retrieval(query: str, top_k_notes: int = 6) -> pd.DataFrame:
            q = vectorizer.transform([query])
            sims = cosine_similarity(q, X).ravel()
            top_idx = np.argsort(-sims)[:top_k_notes]
            rows = []
            for idx in top_idx:
                note = supplier_notes.iloc[int(idx)]
                rows.append({
                    "note_id": note["note_id"],
                    "sku_id": note["sku_id"],
                    "note_text": note["note_text"],
                    "semantic_score": float(sims[idx])
                })
            return pd.DataFrame(rows)

        return "tfidf", semantic_note_retrieval

backend_name, semantic_note_retrieval = build_semantic_backend(supplier_notes)
backend_name


'tfidf'

In [5]:
note_hits = semantic_note_retrieval(question, top_k_notes=8)
note_hits


,note_id,sku_id,note_text,semantic_score
0,N0027,SKU-024,Factory maintenance shutdown planned next week.,0.447214
1,N0021,SKU-018,Factory maintenance shutdown planned next week.,0.447214
2,N0010,SKU-009,Factory maintenance shutdown planned next week.,0.447214
3,N0001,SKU-002,Carrier congestion at port; containers delayed by 4 days.,0.000000
4,N0028,SKU-025,No issues reported; stable supply.,0.000000
5,N0026,SKU-022,Quality hold on latest batch; inspection pending.,0.000000
6,N0025,SKU-021,Supplier reported a raw material shortage affecting lead times.,0.000000
7,N0024,SKU-020,Partial shipment confirmed; remaining units backordered.,0.000000


## 4) Hybrid retrieval fusion

In [6]:
def hybrid_retrieval(question: str, features: pd.DataFrame, top_k: int = 10):
    rule_df = rule_retrieval(features, question, top_k=len(features)).copy()
    note_hits = semantic_note_retrieval(question, top_k_notes=10)

    semantic_by_sku = (
        note_hits.groupby("sku_id")["semantic_score"]
        .max()
        .reset_index()
        .rename(columns={"semantic_score": "retrieval_score_semantic"})
    )

    out = rule_df.merge(semantic_by_sku, on="sku_id", how="left")
    out["retrieval_score_semantic"] = out["retrieval_score_semantic"].fillna(0.0)

    out["retrieval_score_rule_norm"] = out["retrieval_score_rule"] / max(out["retrieval_score_rule"].max(), 1.0)
    out["retrieval_score_semantic_norm"] = out["retrieval_score_semantic"] / max(out["retrieval_score_semantic"].max(), 1e-6)

    out["retrieval_score_hybrid"] = (
        0.75 * out["retrieval_score_rule_norm"] +
        0.25 * out["retrieval_score_semantic_norm"]
    )

    cols = [
        "sku_id","days_of_cover","max_delay_days","demand_spike_pct","buyer_priority",
        "stockout_risk_flag_7d","retrieval_score_rule","retrieval_score_semantic","retrieval_score_hybrid"
    ]
    return out.sort_values("retrieval_score_hybrid", ascending=False).head(top_k)[cols], note_hits

hybrid_top, note_hits = hybrid_retrieval(question, features, top_k=10)
hybrid_top


,sku_id,days_of_cover,max_delay_days,demand_spike_pct,buyer_priority,stockout_risk_flag_7d,retrieval_score_rule,retrieval_score_semantic,retrieval_score_hybrid
0,SKU-007,2.1,10,10,critical,1,129.6,0.000000,0.750000
1,SKU-006,0.8,8,0,critical,1,122.4,0.000000,0.708333
2,SKU-015,1.9,5,50,normal,1,112.0,0.000000,0.648148
11,SKU-009,4.0,8,0,high,0,47.4,0.447214,0.524306
3,SKU-005,1.4,0,0,normal,1,90.0,0.000000,0.520833
12,SKU-018,63.9,8,75,critical,0,44.4,0.447214,0.506944
18,SKU-024,7.5,11,0,critical,0,38.0,0.447214,0.469907
4,SKU-030,2.0,8,35,normal,0,68.0,0.000000,0.393519
5,SKU-026,0.9,14,0,normal,0,68.0,0.000000,0.393519
6,SKU-012,1.9,8,0,high,0,67.4,0.000000,0.390046


## 5) Deterministic risk engine

In [7]:
def compute_risk_score(row: pd.Series):
    components = {}

    if row["days_of_cover"] < 3:
        components["low_cover"] = 35
    elif row["days_of_cover"] < 7:
        components["low_cover"] = 22
    elif row["days_of_cover"] < 14:
        components["low_cover"] = 8
    else:
        components["low_cover"] = 0

    components["shipment_delay"] = min(float(row["max_delay_days"]) * 4.0, 28)
    components["demand_spike"] = min(float(row["demand_spike_pct"]) * 0.5, 30)

    gap = float(row["net_available_7d"] - row["safety_stock_units"])
    components["below_safety_after_7d"] = (
        min(abs(gap) / max(1, row["safety_stock_units"]) * 35, 35) if gap < 0 else 0
    )

    score = max(0.0, min(100.0, float(sum(components.values()))))
    return round(score, 1), components

def risk_level_from_score(score: float) -> str:
    if score >= 70:
        return "high"
    if score >= 40:
        return "medium"
    return "low"

# Score the hybrid candidates
hybrid_full = hybrid_top.merge(features[["sku_id","on_hand_units","avg_daily_demand","safety_stock_units","net_available_7d"]], on="sku_id", how="left")
scores = []
for _, row in hybrid_full.iterrows():
    s, comps = compute_risk_score(row)
    scores.append((s, risk_level_from_score(s), comps))

hybrid_full["risk_score"] = [s for s,_,_ in scores]
hybrid_full["risk_level"] = [lvl for _,lvl,_ in scores]
hybrid_full["risk_components"] = [c for _,_,c in scores]

hybrid_full[["sku_id","risk_score","risk_level","days_of_cover","max_delay_days","demand_spike_pct","net_available_7d"]]


,sku_id,risk_score,risk_level,days_of_cover,max_delay_days,demand_spike_pct,net_available_7d
0,SKU-007,100.0,high,2.1,10,10,-135
1,SKU-006,98.0,high,0.8,8,0,-194
2,SKU-015,100.0,high,1.9,5,50,-44
3,SKU-009,50.0,medium,4.0,8,0,284
4,SKU-005,70.0,high,1.4,0,0,-171
5,SKU-018,58.0,medium,63.9,8,75,602
6,SKU-024,36.0,low,7.5,11,0,671
7,SKU-030,80.5,high,2.0,8,35,408
8,SKU-026,63.0,medium,0.9,14,0,177
9,SKU-012,63.0,medium,1.9,8,0,381


## 6) LLM wrapper + JSON validation + safe fallback

In [8]:
import json
import re
from typing import Set

ALLOWED_RISK_LEVELS: Set[str] = {"low", "medium", "high"}

def extract_json_from_text(text: str) -> Optional[dict]:
    # best-effort: find the first JSON object
    if not isinstance(text, str) or not text.strip():
        return None
    m = re.search(r"\{.*\}", text, flags=re.DOTALL)
    if not m:
        return None
    try:
        return json.loads(m.group(0))
    except Exception:
        return None

def validate_output(record: dict) -> List[str]:
    errors = []
    required = ["sku_id", "risk_score", "risk_level", "top_reasons", "recommended_action", "source_evidence"]

    for key in required:
        if key not in record:
            errors.append(f"Missing field: {key}")

    if "risk_level" in record and record["risk_level"] not in ALLOWED_RISK_LEVELS:
        errors.append("Invalid risk_level")

    if "top_reasons" in record and (not isinstance(record["top_reasons"], list) or len(record["top_reasons"]) == 0):
        errors.append("top_reasons must be non-empty list")

    if "source_evidence" in record and (not isinstance(record["source_evidence"], list) or len(record["source_evidence"]) == 0):
        errors.append("source_evidence must be non-empty list")

    return errors

def build_source_evidence(sku_id: str) -> List[str]:
    # Minimal evidence payload for demo (structured + any supplier note)
    row = features.loc[features["sku_id"] == sku_id].iloc[0]
    ev = [
        f"days_of_cover={float(row['days_of_cover'])}",
        f"max_delay_days={int(row['max_delay_days'])}",
        f"demand_spike_pct={int(row['demand_spike_pct'])}",
        f"net_available_7d={int(row['net_available_7d'])}",
        f"buyer_priority={row['buyer_priority']}",
    ]
    # add at most one note snippet if available
    if not supplier_notes.empty:
        notes = supplier_notes.loc[supplier_notes["sku_id"] == sku_id]["note_text"].head(1).tolist()
        if notes:
            ev.append(f"note: {notes[0]}")
    return ev

def mock_llm_explainer(row: pd.Series, risk_score: float, risk_level: str) -> Dict[str, Any]:
    reasons = []
    if row["days_of_cover"] < 7:
        reasons.append(f"Low cover: {row['days_of_cover']} days")
    if row["max_delay_days"] >= 5:
        reasons.append(f"Shipment delay: {int(row['max_delay_days'])} days")
    if row["demand_spike_pct"] >= 20:
        reasons.append(f"Demand spike: {int(row['demand_spike_pct'])}%")
    if row["net_available_7d"] < row["safety_stock_units"]:
        reasons.append("Projected below safety stock in 7 days")

    if not reasons:
        reasons = ["No single dominant risk factor; monitor trend signals"]

    action = "Review inbound shipments and consider expediting or reallocating safety stock."

    return {
        "sku_id": row["sku_id"],
        "risk_score": float(risk_score),
        "risk_level": risk_level,
        "top_reasons": reasons[:4],
        "recommended_action": action,
        "source_evidence": build_source_evidence(row["sku_id"]),
    }

PROMPT_TEMPLATE = """You are a supply chain copilot.

Given operational evidence and a deterministic risk score, produce ONLY valid JSON with:
- sku_id
- risk_score
- risk_level
- top_reasons (2 to 4 strings)
- recommended_action (1 sentence)
- source_evidence (copied from input)

Rules:
- Do not invent facts
- Use only provided evidence
- Output JSON only
"""

def build_generation_prompt(payload: Dict[str, Any]) -> str:
    return PROMPT_TEMPLATE + "\n\nINPUT:\n" + json.dumps(payload, indent=2)

def call_ollama_llama(prompt: str, model: str = "llama3.2", timeout_sec: int = 30) -> str:
    import requests
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {"temperature": 0.1, "num_predict": 256},
    }
    r = requests.post("http://localhost:11434/api/generate", json=payload, timeout=timeout_sec)
    r.raise_for_status()
    return r.json().get("response", "")

def generate_structured_summary(row: pd.Series, risk_score: float, risk_level: str, backend: str = "mock") -> Dict[str, Any]:
    fallback = mock_llm_explainer(row, risk_score, risk_level)

    if backend == "mock":
        return fallback

    # Build context payload
    payload = {
        "sku_id": row["sku_id"],
        "deterministic_risk_score": float(risk_score),
        "deterministic_risk_level": risk_level,
        "structured_features": {
            "days_of_cover": float(row["days_of_cover"]),
            "max_delay_days": int(row["max_delay_days"]),
            "demand_spike_pct": int(row["demand_spike_pct"]),
            "net_available_7d": int(row["net_available_7d"]),
            "buyer_priority": row["buyer_priority"],
        },
        "source_evidence": build_source_evidence(row["sku_id"]),
    }

    try:
        raw = call_ollama_llama(build_generation_prompt(payload))
        parsed = extract_json_from_text(raw)
        if parsed is None or validate_output(parsed):
            return fallback

        # enforce deterministic values
        parsed["sku_id"] = row["sku_id"]
        parsed["risk_score"] = float(risk_score)
        parsed["risk_level"] = risk_level
        parsed["source_evidence"] = build_source_evidence(row["sku_id"])
        return parsed

    except Exception:
        return fallback


## 7) End-to-end run (top 5 SKUs)

In [9]:
top5 = hybrid_full.sort_values(["risk_score","retrieval_score_hybrid"], ascending=False).head(5).copy()

records = []
for _, row in top5.iterrows():
    s = float(row["risk_score"])
    lvl = row["risk_level"]
    rec = generate_structured_summary(row, s, lvl, backend="mock")
    records.append(rec)

pd.DataFrame(records)[["sku_id","risk_score","risk_level","top_reasons","recommended_action"]]


,sku_id,risk_score,risk_level,top_reasons,recommended_action
0,SKU-007,100.0,high,"[Low cover: 2.1 days, Shipment delay: 10 days, Projected below safety stock ...",Review inbound shipments and consider expediting or reallocating safety stock.
1,SKU-015,100.0,high,"[Low cover: 1.9 days, Shipment delay: 5 days, Demand spike: 50%, Projected b...",Review inbound shipments and consider expediting or reallocating safety stock.
2,SKU-006,98.0,high,"[Low cover: 0.8 days, Shipment delay: 8 days, Projected below safety stock i...",Review inbound shipments and consider expediting or reallocating safety stock.
3,SKU-030,80.5,high,"[Low cover: 2.0 days, Shipment delay: 8 days, Demand spike: 35%]",Review inbound shipments and consider expediting or reallocating safety stock.
4,SKU-005,70.0,high,"[Low cover: 1.4 days, Projected below safety stock in 7 days]",Review inbound shipments and consider expediting or reallocating safety stock.


In [10]:
# Show evidence payload for the highest-risk SKU
records[0]


{'sku_id': 'SKU-007',
 'risk_score': 100.0,
 'risk_level': 'high',
 'top_reasons': ['Low cover: 2.1 days',
  'Shipment delay: 10 days',
  'Projected below safety stock in 7 days'],
 'recommended_action': 'Review inbound shipments and consider expediting or reallocating safety stock.',
 'source_evidence': ['days_of_cover=2.1',
  'max_delay_days=10',
  'demand_spike_pct=10',
  'net_available_7d=-135',
  'buyer_priority=critical',
  'note: Carrier congestion at port; containers delayed by 6 days.']}